In [1]:
import hydra
import hydra.experimental
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer
from omegaconf import DictConfig

from model import Seq2Seq
from configure_dataloader import DataLoaderHandler
from torchfly.text.decode import TransformerDecoder
from torchfly.common import set_random_seed

In [2]:
set_random_seed(123)

In [3]:
hydra.experimental.initialize("config")
config = hydra.experimental.compose("config.yaml")

In [4]:
config.task.data_dir

'/home/wuqy1203/Desktop/Projects/TextGAIL/data/CommonGEN'

In [5]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

In [6]:
dataloader_handler = DataLoaderHandler(config)

In [7]:
train_dataloader = dataloader_handler.train_dataloader(config)

In [8]:
model = Seq2Seq(config)

File exists: /home/wuqy1203/.cache/torchfly/models/roberta-tokenized-gpt2.pth
_IncompatibleKeys(missing_keys=['lm_head.weight'], unexpected_keys=['lm_head.decoder.weight'])


In [9]:
model_weights = torch.load("/home/wuqy1203/Desktop/Projects/TextGAIL/Experiments/MLE/outputs/CommonGEN/Checkpoints/iter_2439_model_state.pth")
model.load_state_dict(model_weights, strict=False)

<All keys matched successfully>

In [10]:
model = model.cuda()

In [11]:
# Register model with TransformerDecoder
model.register_generator(model.decoder)
model.register_tokenizer(model.tokenizer)
model.setup_config(config.decode)

In [12]:
batch = next(iter(train_dataloader))

In [13]:
results = model.generate(input_ids=batch['source_token_ids'].cuda(), num_beams=4)

In [14]:
results = model.generate(input_ids=batch['source_token_ids'].cuda(), num_beams=2)

In [42]:
results = model.generate(input_ids=batch['source_token_ids'].cuda(), num_beams=4, num_return_sequences=2)

In [56]:
results = model.generate(input_ids=batch['source_token_ids'].cuda(), num_beams=1)

In [57]:
batch_idx = 0
print(tokenizer.decode(batch['source_token_ids'][batch_idx].tolist()))
print(tokenizer.decode(results["tokens"][batch_idx][0].tolist()))

silhouette_N#symbol_N#vector_N<pad><pad><pad><pad>
<s>symbol of the harmonious cross on the black silhouette with symbol on white background vector</s>
